In [13]:
import cv2 as cv
import numpy as np
import mediapipe as mp
import pandas as pd
import ydf
import numpy as np
import os
import torch
import torch.nn.functional as F 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers=nn.Sequential(
            nn.Linear(len(train.columns)-2,1024),
            nn.ReLU(),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Linear(512,364),
            nn.ReLU(),
            nn.Linear(364,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(64,29),
            # nn.ReLU(),
            # nn.Linear(32,29)
        )
    def forward(self,x):
        logits=self.layers(x)

        return logits

In [5]:
from torch.utils.data import Dataset
class gestureCreator(Dataset):
    def __init__(self,dataframe):
        self.data=dataframe
    def __len__(self):
        return len(self.data)
    def __getitem__(self):
        features=self.data.values.astype(float) 
        features=torch.tensor(features,dtype=torch.float32)
        return features   

In [6]:
m=torch.load('model_f.pt')
m.eval()
device = torch.device('cpu')


C:\Users\moham\AppData\Local\Temp\ipykernel_11780\1624213714.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  m=torch.load('model_f.pt')


In [7]:
m2=ydf.load_model("model_tree")

In [8]:
mp_h=mp.solutions.hands
hand=mp_h.Hands()
mp_draw=mp.solutions.drawing_utils

In [9]:
gesture_mapper={
    
    0: 'A',
    1: 'B',
    2: 'C',
    3: 'D',
    4: 'del',
    5: 'E',
    6: 'F',
    7: 'G',
    8: 'H',
    9: 'I',
    10: 'J',
    11: 'K',
    12: 'L',
    13: 'M',
    14: 'N',
    15: 'nothing',
    16: 'O',
    17: 'P',
    18: 'Q',
    19: 'R',
    20: 'S',
    21: 'space',
    22: 'T',
    23: 'U',
    24: 'V',
    25: 'W',
    26: 'X',
    27: 'Y',
    28: 'Z'
}

In [10]:
columns=[f'{coord}{i}'for i in range(21) for coord in ['x','y','z']]

In [16]:
def control():
    cap=cv.VideoCapture(0)
    while True:
        ret,frame=cap.read()

        cv.flip(frame,1)
        frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        result = hand.process(frame_rgb)
        if result.multi_hand_landmarks:
            for hand_index, hand_landmarks in enumerate(result.multi_hand_landmarks):
                handedness = result.multi_handedness[hand_index].classification[0].label
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_h.HAND_CONNECTIONS)
                land = []
                land2=[hand_index]

                for lm in hand_landmarks.landmark:

                    land.extend([lm.x, lm.y, lm.z])

                    land2.extend([lm.x, lm.y, lm.z])

    

                df=pd.DataFrame([land],columns=columns)
                df = df.astype(float)
                print(m2.data_spec())

                print(df)
                #pred=m2.predict(df)
                print(df.dtypes)

# Check unique values for categorical columns
                print(df['x0'].unique())

                

                features=torch.tensor(land,dtype=torch.float32).unsqueeze(0).to(device)

                with torch.no_grad():

                    out=m(features)

                    _,val=torch.max(out,1)

                    pred=val.item()

                    label=gesture_mapping.get(pred,"Unknown")

                #avg=(out.cpu().numpy()+pred)/2

                #pred_ensemble=np.argmax(avg,axis=1)[0]

                #label_ensemble=gesture_mapping.get(pred_ensemble,"Unkown")

                wrist_pos=hand_landmarks.landmark[0]

                h,w,_=frame.shape

                wrist_x=int(wrist_pos.x*w)

                wrist_y=int(wrist_pos.y*h)

                pos=(wrist_x,wrist_y-30)




                cv.putText(

                    frame, 

                    f'{label}', 

                    pos, 

                    cv.FONT_HERSHEY_SIMPLEX, 

                    1, 

                    (255, 0, 0), 

                    2, 

                    cv.LINE_AA

                )

                tasks(label_ensemble)




        # Display the image 

        cv.imshow('feed', cv.flip(frame, 1))

        if cv.waitKey(1) == ord("q"):

            cap.release()

            cv.destroyAllWindows()

            break

In [18]:
control()

columns {
  type: CATEGORICAL
  name: "gesture"
  categorical {
    number_of_unique_values: 3
    items {
      key: "Right"
      value {
        index: 2
        count: 327
      }
    }
    items {
      key: "Left"
      value {
        index: 1
        count: 11480
      }
    }
    items {
      key: "<OOD>"
      value {
        index: 0
        count: 0
      }
    }
  }
  count_nas: 0
  dtype: DTYPE_BYTES
}
columns {
  type: CATEGORICAL
  name: "x0"
  categorical {
    number_of_unique_values: 1
    items {
      key: "<OOD>"
      value {
        index: 0
        count: 11807
      }
    }
  }
  count_nas: 0
  dtype: DTYPE_BYTES
}
columns {
  type: CATEGORICAL
  name: "x1"
  categorical {
    number_of_unique_values: 1
    items {
      key: "<OOD>"
      value {
        index: 0
        count: 11807
      }
    }
  }
  count_nas: 0
  dtype: DTYPE_BYTES
}
columns {
  type: CATEGORICAL
  name: "x2"
  categorical {
    number_of_unique_values: 1
    items {
      key: "<OOD>"


c:\Users\moham\anaconda3\envs\JIT\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


ValueError: Cannot import column 'x0' with semantic=Semantic.CATEGORICAL as it contains floating point values.
Got array([0.64902538]).